In [2]:
from prelu_cnn import CNN

from datasets import load_from_disk

# Load training dataset
dataset_path = "../processed_datasets/imagenet_processor"
ds = load_from_disk(dataset_path)
ds = ds['train'].select(range(1000))





Loading dataset from disk:   0%|          | 0/303 [00:00<?, ?it/s]

In [3]:
ds['pixel_values'].shape, type(ds['pixel_values'])

(torch.Size([1000, 3, 224, 224]), torch.Tensor)

In [38]:
inputs = ds['pixel_values']
labels = ds['labels']

In [39]:
import torch
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move model and inputs to GPU if available
model = CNN(use_prelu=False, use_builtin_conv=True).to(device)

# Move tensors to the same device as the model
inputs = inputs.to(device)
labels = labels.to(device)

outputs = model(inputs)
outputs

Using device: cuda


tensor([[-0.1992, -0.0423,  0.0543,  ...,  0.1351,  0.0930, -0.2633],
        [ 0.9272,  0.3014,  0.3716,  ...,  0.1198,  0.0743, -0.0290],
        [-0.1074,  0.0388,  0.1548,  ...,  0.4969, -0.4893,  0.4166],
        ...,
        [ 0.4961,  0.1919,  0.6859,  ..., -0.3899, -0.2643,  0.3357],
        [ 0.2203, -0.2539, -0.1453,  ...,  0.4844,  0.2236, -0.0192],
        [-0.1353,  0.0568,  0.1968,  ...,  0.1761, -0.0790,  0.5020]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [40]:
import numpy as np

# axis=1 means we're finding the maximum value along the second dimension (columns)
# For a 2D tensor with shape (batch_size, num_classes), axis=1 gives us the class index
# with the highest probability for each sample in the batch
predictions = np.argmax(outputs.detach().cpu().numpy(), axis=1)
print("Predictions:", predictions)
print("Labels:", labels.cpu().numpy())
print("Correct predictions:", (predictions == labels.cpu().numpy()).sum(), "out of", len(predictions))


Predictions: [426 186 596 596 417 815 528 603 799  71 874 320 444 857 417 945  71 333
 298 895 974 609 525 134 246 760 241 136 926 475 188 709 763  71 510 759
 709 892 456 188 804 759  71 926 282  22  71 601 778 484  30 282 513 929
 458 164  22 943 881 329 505 804 321  27 393 408 488 804 282  72 298 596
 436 645 596 228 596 303 902 748 536 988 380 428 332 580 444  71 878 473
 293  64 645 530 530 645 892 845 596 329 528 702 337 853 829  71 809 333
 128 507 702 596 337 596 298 955 846 767 596 596 870 265  59 163 279 134
 246 870 898 444 899 365 384 965  71 343 671  71 134  64  71 417 702 436
 566 890 804 804 496 945 596 417 346 699 943  54 666 945  48  21 926 596
 266 528 738 346 174 709 164 381  35 382 701 128 340 338 589 702 512 417
 517 382 473  41 302 703 857  71 332 926 870 545 943 596 293 945 296 473
 188 867 382 326 436 189 365 702 444  71 403 572 783 597 478 596 282 804
 701 964 865 550 892 213  58 545  64 673 721 943 338 401 128 837 709 596
 709 596 526 627 654 596  58 362 945  

In [41]:
import torch.nn as nn

loss_fct = nn.CrossEntropyLoss()
loss = loss_fct(outputs, labels)
loss

tensor(6.9683, device='cuda:0', grad_fn=<NllLossBackward0>)